In [1]:
import numpy as np
import nibabel as nib
from nilearn.image import resample_img


In [2]:
base_fn = "/home/lmahler/data/scratch/fastsurfer_val/el4879_fastsurfer.nii"
obj = nib.load(base_fn)

In [3]:
img = obj.get_fdata()
np.unique(img.astype(int))

array([   0,    2,    4,    5,    7,    8,   10,   11,   12,   13,   14,
         15,   16,   17,   18,   24,   26,   28,   31,   41,   43,   44,
         46,   47,   49,   50,   51,   52,   53,   54,   58,   60,   63,
         77, 1002, 1003, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012,
       1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023,
       1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1034, 1035, 2002,
       2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025,
       2026, 2027, 2028, 2029, 2030, 2031, 2034, 2035])

In [4]:
def aparaseg_to_aseg(img):
    img[img >= 2000] = 42
    img[img >= 1000] = 3
    return img


In [5]:
new = aparaseg_to_aseg(img)
np.unique(new)

array([ 0.,  2.,  3.,  4.,  5.,  7.,  8., 10., 11., 12., 13., 14., 15.,
       16., 17., 18., 24., 26., 28., 31., 41., 42., 43., 44., 46., 47.,
       49., 50., 51., 52., 53., 54., 58., 60., 63., 77.])

In [6]:
def aseg_to_fs_gmwm(img):
    img = img.astype(int)
    new = np.zeros(img.shape, dtype=int)
    new = np.where(img==42, 42, new)
    new = np.where(img==3, 3, new)
    new = np.where(img==41, 41, new)
    new = np.where(img==2, 2, new)
    return new



In [7]:
new = aseg_to_fs_gmwm(new)
np.unique(new)

array([ 0,  2,  3, 41, 42])

In [8]:
test_obj = nib.load("/home/lmahler/data/scratch/fastsurfer_val/out/el4879_fastsurfer.nii")

In [9]:
test_img = test_obj.get_fdata()
np.unique(test_img)

array([ 0.,  2., 41.])

In [10]:
test_img = test_img.astype(int)
test_img.dtype

dtype('int64')

In [11]:
hdr = test_obj.header
hdr.set_data_dtype(test_img.dtype)
new_obj = nib.nifti1.Nifti1Image(test_img, affine=test_obj.affine, header=hdr)

In [12]:
np.unique(new_obj.get_fdata())

array([ 0.,  2., 41.])

In [13]:
test_img.dtype

dtype('int64')

In [14]:
new_obj.header.get_data_dtype()

dtype('<i8')

In [15]:
dat = new_obj.get_fdata()

In [16]:
dat.dtype

dtype('float64')

In [17]:
np.unique(dat)

array([ 0.,  2., 41.])

In [18]:
nib.save(new_obj, "/tmp/test.nii")

In [19]:
load_test = nib.load("/tmp/test.nii")

In [20]:
load_dat = load_test.get_fdata()

In [21]:
load_dat.dtype

dtype('float64')

In [22]:
np.unique(load_dat)

array([ 0.,  2., 41.])

In [23]:
comp = nib.load("/home/lmahler/data/GMWM_Segmentation9t/el4879/el4879_seg/el4879_seg.nii")

In [24]:
res = resample_img(load_test, target_affine=comp.affine, target_shape=comp.shape, interpolation="nearest")

In [25]:
np.unique(res.get_fdata())

array([ 0.,  2., 41.])

In [26]:
import pandas as pd

In [27]:
df = pd.read_csv("/home/lmahler/code/mpi_val/results/fastsurfer_results.csv")

In [28]:
df

,Unnamed: 0,Name,Dice,AverageSurfaceDistance,Hausdorff
0,0,210525,0.178056,"[(34.58133500193655, 4.121080584285436), (31.5...","[98.812954616285, 95.38343671728336]"
1,1,l4879,0.149874,"[(28.356675697398444, 9.45788156762123), (24.1...","[77.20751258783046, 73.79024325749306]"
2,2,l7349,0.218953,"[(35.098543594323154, 3.997058368851568), (32....","[96.82974749528164, 96.13012014972206]"
3,3,fb9364,0.248825,"[(27.12997857961704, 1.6521785988714803), (25....","[86.4060183089118, 86.95976080923866]"
4,4,fb9455,0.113524,"[(26.15624982769581, 6.7939638633955495), (24....","[84.09518416651456, 80.51086883148137]"
5,5,jb1922,0.178845,"[(14.344039881631957, 3.6536612826712234), (18...","[40.07492981902776, 58.89821729050889]"
6,6,me0801,0.263014,"[(26.708711881169595, 1.553720116189858), (24....","[85.0, 84.65222974027323]"
7,7,me1102,0.176611,"[(33.30060406654836, 5.73763383868492), (29.33...","[81.36952746575342, 77.17512552629896]"
8,8,vk1506,0.089678,"[(61.59717550333535, 17.527537664007298), (57....","[126.68859459319927, 120.47406359876801]"
9,9,vk3464,0.226255,"[(23.540393376110917, 2.742309898027691), (19....","[76.90253571892151, 73.00684899377592]"
